# Emotion Detection from YouTube videos using `fer`

## Helper functions

In [ ]:
def cut_video(video,fr,to):
  video_edit = VideoFileClip(video).subclip(fr,to)
  video_edit.write_videofile('temp/temp_video_edit.mp4',preset='ultrafast',audio=False)
  return 'temp/temp_video_edit.mp4'

def frame_frequency(vf,total_frames=400):
  cap=cv2.VideoCapture(vf)
  fps = cap.get(cv2.CAP_PROP_FPS)
  frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
  duration = frame_count/fps
  print('fps = ' + str(fps))
  print('number of frames = ' + str(frame_count))
  print('duration (S) = ' + str(duration))
  freq = math.floor(frame_count/(total_frames))
  return freq

def detect_emotions(vf,i, mode='freq',freq=50):
  video = Video(vf)
  video.first_face_only = False
  if mode=='freq':
    raw_data = video.analyze(detector, display=False, frequency = freq, save_video = False, annotate_frames = True, zip_images = True)
    tdf = video.to_pandas(raw_data)
    tdf.to_csv('../data-clean/%s-f%s.csv' %(i, str(freq)))
    os.system('cp output/images.zip ../output/%s-f%s.zip' %(i,str(freq)))
  else:
    ff = frame_frequency(vf,total_frames=freq)
    raw_data = video.analyze(detector, display=False, frequency = ff, save_video = False, annotate_frames = True, zip_images = True)
    tdf = video.to_pandas(raw_data)
    tdf.to_csv('../data-clean/%s-t%s.csv' %(i, str(freq)))
    os.system('cp output/images.zip ../output/%s-t%s.zip' %(i,str(freq)))
  return tdf

def process_vid(csv,i, mode = 'freq', freq = 50):
  yt = YouTube(csv['url'][i])
  yt_vid = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()
  yt_vid.download('temp/')
  src = os.listdir('temp/')[0]
  os.rename('temp/'+src,'temp/temp_video.mp4')
  print("File renamed!")
  videofile = 'temp/temp_video.mp4'
  if (csv['crop'][i] == 'yes'):
    videofile = cut_video(videofile,fr=csv["start"][i],to=csv["end"][i])
  return detect_emotions(videofile,i,mode,freq)
  


## Setup

In [ ]:
# install requirements
!pip install -q fer
!pip install imageio==2.4.1
!pip install -q pytube
# import libraries
import tensorflow as tf
from pytube import YouTube
import os
from fer import Video
from fer import FER
import matplotlib.pyplot as plt
import sys
import pandas as pd
import cv2
import math
import numpy as np
from moviepy.editor import *
# notebook setings
%matplotlib inline
from pylab import rcParams
rcParams['figure.figsize'] = 10, 8

detector = FER(mtcnn=True)


## Graphic card test

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
     

## Clear previous results

---



In [ ]:
!rm -rf '../output/'
!rm -rf 'temp/'


# Analyze videos

In [5]:
import pandas as pd
import os

In [7]:
url_df = pd.read_csv('../data-raw/yt-urls.csv') 
os.mkdir('temp')
n = len(url_df)

In [ ]:
# Main processing method
# mode = 'total', total 300 frames

print("Starting main processing")
print("Each video - 300 frames in total")

for k in range(n):
  !rm -rf 'output/'
  !rm -rf 'temp/'
  print('Video number #%s \n' %(k))
  print('Url: %s' %(url_df['url'][k]))
  temp = process_vid(url_df,k, mode = 'total', freq=300)
  print('Video number #%s complete! Moving on ... \n' %(k))


In [ ]:
# Alternative processing method
# mode = 'freq', every 50th frame

print("Starting alternative processing")
print("Each video - 300 frames in total")

for k in range(n):
  !rm -rf 'output/'
  !rm -rf 'temp/'
  print('Video number #%s \n' %(k))
  print('Url: %s' %(url_df['url'][k]))
  temp = process_vid(url_df,k, mode = 'freq', freq=50)
  print('Video number #%s complete! Moving on ... \n' %(k))
